# Langchain

El objetivo de este post es explicar Langchain, pero además usando los servición más importantes de IA de la actualidad, que son los de Hugging Face y que no entiendo por qué no aparece en la documentación de Langchain.

## Instalación

Para instalar Langchain podemos hacerlo con Conda:

``` bash
conda install langchain -c conda-forge
```

o con Pip:

``` bash
pip install langchain
```

Ahora tenemos que instalar la integración de Hugging Face:

``` bash
pip install langchain-huggingface
```

## Token de Hugging Face

Para poder usar la `API Inference` de HuggingFace, lo primero que necesitas es tener una cuenta en HuggingFace. Una vez la tengas, hay que ir a [Access tokens](https://huggingface.co/settings/keys) en la configuración de tu perfil y generar un nuevo token.

Hay que ponerle un nombre. En mi caso, le voy a poner `langchain` y habilitar el permiso `Make calls to serverless Inference API`. Nos creará un token que tendremos que copiar

Para gestionar el token, vamos a crear un archivo en la misma ruta en la que estemos trabajando llamado`.env` y vamos a poner el token que hemos copiado en el archivo de la siguiente manera:

``` bash
HUGGINGFACE_TOKEN="hf_...."
```

Ahora, para poder obtener el token, necesitamos tener instalado `dotenv`, que lo instalamos mediante

```bash
pip install python-dotenv
```

Y ejecutamos lo siguiente

In [2]:
import os
import dotenv

dotenv.load_dotenv()

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

Ahora que tenemos un token, creamos un cliente. Para ello, necesitamos tener instalada la librería `huggingface_hub`. La instalamos mediante conda o pip.

``` bash
conda install -c conda-forge huggingface_hub
```

o

```bash
pip install --upgrade huggingface_hub
```

Ahora tenemos que elegir qué modelo vamos a usar. Puedes ver los modelos disponibles en la página de [Supported models](https://huggingface.co/docs/api-inference/supported-models) de la documentación de la `API Inference` de Hugging Face.

Como a la hora de escribir el post, el mejor disponible es `Qwen2.5-72B-Instruct`, vamos a usar ese modelo.

In [3]:
MODEL = "Qwen/Qwen2.5-72B-Instruct"

Ahora podemos crear el cliente

In [4]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_TOKEN, model=MODEL)
client

<InferenceClient(model='Qwen/Qwen2.5-72B-Instruct', timeout=None)>

Hacemos una prueba a ver si funciona

In [5]:
message = [
	{ "role": "user", "content": "Hola, qué tal?" }
]

stream = client.chat.completions.create(
	messages=message, 
	temperature=0.5,
	max_tokens=1024,
	top_p=0.7,
	stream=False
)

response = stream.choices[0].message.content
print(response)

¡Hola! Estoy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?


# Crear un sencillo LLM

In [9]:
from langchain_huggingface import HuggingFaceEndpoint

model = HuggingFaceEndpoint(model="Qwen/Qwen2.5-72B-Instruct", huggingfacehub_api_token=HUGGINGFACE_TOKEN)

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi!"),
]

response = model.invoke(messages)
response.content

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct (Request ID: hiYlCT)

TooManyRequests: Please log in or use a HF access token